# Nettoyage et restructuration des données de matières premières

Ce notebook montre comment nettoyer et restructurer un DataFrame contenant des données de matières premières avec plusieurs colonnes de dates et de prix.

## Import Required Libraries
Import pandas and other necessary libraries for data manipulation.

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Chargement des données

In [37]:
# Chargement du fichier CSV
df = pd.read_csv('DATA.csv', sep=';', encoding='utf-8')

# Affichage des premières lignes pour vérifier la structure
print("Forme du DataFrame:", df.shape)
df.head()

Forme du DataFrame: (4019, 24)


,DATEBrent,Brent,DATECrudeOilWTI,CrudeOilWTI,DATEGold,Gold,DATEHenryHub,HenryHub,DATEWheat,Wheat,...,DATELiveCattle,LiveCattle,DATEFeederCattle,FeederCattle,DATECoffee,Coffee,DATECotton,Cotton,DATESugar,Sugar
0,01/04/2010,80.12,01/04/2010,81.51,01/04/2010,1118.3,01/04/2010,5.884,01/04/2010,0.1775,...,01/04/2010,85.375,01/04/2010,95.875,01/04/2010,141.85,01/04/2010,76.00,01/04/2010,27.62
1,01/05/2010,80.59,01/05/2010,81.77,01/05/2010,1118.7,01/05/2010,5.637,01/05/2010,0.1600,...,01/05/2010,86.325,01/05/2010,96.550,01/05/2010,141.00,01/05/2010,73.12,01/05/2010,27.64
2,01/06/2010,81.89,01/06/2010,83.18,01/06/2010,1136.5,01/06/2010,6.009,01/06/2010,0.1350,...,01/06/2010,85.950,01/06/2010,96.300,01/06/2010,141.60,01/06/2010,73.55,01/06/2010,28.41
3,01/07/2010,81.51,01/07/2010,82.66,01/07/2010,1133.7,01/07/2010,5.806,01/07/2010,0.1525,...,01/07/2010,85.925,01/07/2010,96.450,01/07/2010,141.90,01/07/2010,72.89,01/07/2010,28.00
4,01/08/2010,81.37,01/08/2010,82.75,01/08/2010,1138.9,01/08/2010,5.749,01/08/2010,0.1550,...,01/08/2010,85.825,01/08/2010,96.350,01/08/2010,145.35,01/08/2010,72.44,01/08/2010,27.53


## 2. Analyse de la structure actuelle

In [38]:
# Affichage des colonnes
print("Colonnes du DataFrame:")
df.columns

Colonnes du DataFrame:


Index(['DATEBrent', 'Brent', 'DATECrudeOilWTI', 'CrudeOilWTI', 'DATEGold',
       'Gold', 'DATEHenryHub', 'HenryHub', 'DATEWheat', 'Wheat',
       'DATESoybeans', 'Soybeans', 'DATEHeatingOil', 'HeatingOil',
       'DATELiveCattle', 'LiveCattle', 'DATEFeederCattle', 'FeederCattle',
       'DATECoffee', 'Coffee', 'DATECotton', 'Cotton', 'DATESugar', 'Sugar'],
      dtype='object')

## 3. Restructuration des données

La stratégie sera de :
1. Créer un nouveau DataFrame propre avec une seule colonne de date
2. Ajouter chaque matière première en harmonisant les dates

In [39]:
# Première approche : vérifier si les dates sont alignées entre les différentes matières premières
# Prenons quelques exemples pour comparer
date_cols = [col for col in df.columns if 'DATE' in col]
price_cols = [col for col in df.columns if 'DATE' not in col]

# Vérification de l'égalité des dates pour quelques colonnes
if len(df) > 0:
    print("Nombre de valeurs uniques par colonne de date:")
    for col in date_cols:
        print(f"{col}: {df[col].nunique()}")
    
    # Vérifions si les premières dates correspondent
    print("\nPremières dates de chaque colonne:")
    for col in date_cols:
        print(f"{col}: {df[col].iloc[0]}")

Nombre de valeurs uniques par colonne de date:
DATEBrent: 3873
DATECrudeOilWTI: 3779
DATEGold: 3779
DATEHenryHub: 3779
DATEWheat: 2693
DATESoybeans: 4019
DATEHeatingOil: 3779
DATELiveCattle: 3779
DATEFeederCattle: 3780
DATECoffee: 3776
DATECotton: 3776
DATESugar: 3776

Premières dates de chaque colonne:
DATEBrent: 01/04/2010
DATECrudeOilWTI: 01/04/2010
DATEGold: 01/04/2010
DATEHenryHub: 01/04/2010
DATEWheat: 01/04/2010
DATESoybeans: 01/01/2010
DATEHeatingOil: 01/04/2010
DATELiveCattle: 01/04/2010
DATEFeederCattle: 01/04/2010
DATECoffee: 01/04/2010
DATECotton: 01/04/2010
DATESugar: 01/04/2010


In [40]:
# Méthode 1 : Si les dates sont alignées (même nombre, mêmes valeurs)
def method_aligned_dates():
    # Créer un nouveau DataFrame avec une seule colonne de date (utilisons celle du Brent comme référence)
    clean_df = pd.DataFrame()
    clean_df['Date'] = pd.to_datetime(df['DATEBrent'])
    
    # Ajouter les colonnes de prix en conservant leurs noms d'origine
    for col in price_cols:
        clean_df[col] = df[col]
    
    # Définir la date comme index
    clean_df = clean_df.set_index('Date')
    
    return clean_df

# Méthode 2 : Si les dates NE sont PAS alignées
def method_unaligned_dates():
    # Créer des DataFrames individuels pour chaque matière première
    commodities = {}
    
    # Liste des paires (date, prix) de commodités
    commodity_pairs = [
        ('DATEBrent', 'Brent'),
        ('DATECrudeOilWTI', 'CrudeOilWTI'),
        ('DATEGold', 'Gold'),
        ('DATESilver', 'Silver'),
        ('DATEHenryHub', 'HenryHub'),
        ('DATEWheat', 'Wheat'),
        ('DATESoybeans', 'Soybeans'),
        ('DATEHeatingOil', 'HeatingOil'),
        ('DATELiveCattle', 'LiveCattle'),
        ('DATEFeederCattle', 'FeederCattle'),
        ('DATECoffee', 'Coffee'),
        ('DATECotton', 'Cotton'),
        ('DATESugar', 'Sugar')
    ]
    
    # Créer un DataFrame pour chaque matière première
    for date_col, price_col in commodity_pairs:
        # Assurer que la colonne date existe
        if date_col in df.columns and price_col in df.columns:
            temp_df = df[[date_col, price_col]].copy()
            temp_df.columns = ['Date', price_col]
            temp_df['Date'] = pd.to_datetime(temp_df['Date'])
            temp_df = temp_df.dropna()
            temp_df = temp_df.set_index('Date')
            commodities[price_col] = temp_df
    
    # Fusionner tous les DataFrames sur les dates (outer join pour garder toutes les dates)
    clean_df = pd.concat(commodities.values(), axis=1)
    
    return clean_df

In [41]:
# Choix de la méthode selon l'alignement des dates
# Pour déterminer si les dates sont alignées, vérifions si toutes les colonnes DATE ont le même nombre de valeurs uniques
if len(date_cols) > 0 and all(df[date_cols[0]].nunique() == df[col].nunique() for col in date_cols):
    print("Les dates semblent être alignées. Utilisation de la méthode 1.")
    clean_df = method_aligned_dates()
else:
    print("Les dates ne sont pas alignées. Utilisation de la méthode 2.")
    clean_df = method_unaligned_dates()

print("\nStructure du DataFrame nettoyé:")
print(f"Dimensions: {clean_df.shape}")
clean_df.head()

Les dates ne sont pas alignées. Utilisation de la méthode 2.

Structure du DataFrame nettoyé:
Dimensions: (4101, 12)


,Brent,CrudeOilWTI,Gold,HenryHub,Wheat,Soybeans,HeatingOil,LiveCattle,FeederCattle,Coffee,Cotton,Sugar
Date,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,260.8,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-02,NaN,NaN,NaN,NaN,NaN,261.1,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,80.12,81.51,1118.3,5.884,0.1775,270.6,219.05,85.375,95.875,141.85,76.00,27.62
2010-01-05,80.59,81.77,1118.7,5.637,0.1600,263.6,219.41,86.325,96.550,141.00,73.12,27.64
2010-01-06,81.89,83.18,1136.5,6.009,0.1350,274.7,220.32,85.950,96.300,141.60,73.55,28.41


## 4. Vérification et traitement des valeurs manquantes

In [42]:
# Afficher le nombre de valeurs manquantes par colonne
print("Nombre de valeurs manquantes par colonne:")
print(clean_df.isna().sum())

# Pourcentage de valeurs manquantes
print("\nPourcentage de valeurs manquantes par colonne:")
print(clean_df.isna().mean() * 100)

Nombre de valeurs manquantes par colonne:
Brent            228
CrudeOilWTI      322
Gold             322
HenryHub         322
Wheat           1408
Soybeans          82
HeatingOil       322
LiveCattle       322
FeederCattle     321
Coffee           325
Cotton           325
Sugar            325
dtype: int64

Pourcentage de valeurs manquantes par colonne:
Brent            5.559620
CrudeOilWTI      7.851743
Gold             7.851743
HenryHub         7.851743
Wheat           34.333089
Soybeans         1.999512
HeatingOil       7.851743
LiveCattle       7.851743
FeederCattle     7.827359
Coffee           7.924896
Cotton           7.924896
Sugar            7.924896
dtype: float64


## 5. Options de traitement des valeurs manquantes

In [43]:
# Option 1: Supprimer les lignes avec des valeurs manquantes (peut être trop restrictif)
clean_df_dropna = clean_df.dropna()
print(f"Dimensions après suppression des valeurs manquantes: {clean_df_dropna.shape}")

# Option 2: Remplir les valeurs manquantes avec la méthode forward fill
clean_df_ffill = clean_df.fillna(method='ffill')

# Option 3: Remplir les valeurs manquantes avec la méthode backward fill
clean_df_bfill = clean_df.fillna(method='bfill')

# Option 4: Interpolation linéaire
clean_df_interp = clean_df.interpolate(method='linear')

# Choisissons l'option d'interpolation linéaire pour cet exemple
final_df = clean_df_interp.copy()

# Vérification des valeurs manquantes restantes
print(f"Valeurs manquantes restantes: {final_df.isna().sum().sum()}")
final_df.head()

Dimensions après suppression des valeurs manquantes: (2649, 12)
Valeurs manquantes restantes: 22


/var/folders/qk/340w2xz14r9768bjtzw6m6sc0000gn/T/ipykernel_19714/2339907875.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  clean_df_ffill = clean_df.fillna(method='ffill')
/var/folders/qk/340w2xz14r9768bjtzw6m6sc0000gn/T/ipykernel_19714/2339907875.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  clean_df_bfill = clean_df.fillna(method='bfill')


,Brent,CrudeOilWTI,Gold,HenryHub,Wheat,Soybeans,HeatingOil,LiveCattle,FeederCattle,Coffee,Cotton,Sugar
Date,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,260.8,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-02,NaN,NaN,NaN,NaN,NaN,261.1,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,80.12,81.51,1118.3,5.884,0.1775,270.6,219.05,85.375,95.875,141.85,76.00,27.62
2010-01-05,80.59,81.77,1118.7,5.637,0.1600,263.6,219.41,86.325,96.550,141.00,73.12,27.64
2010-01-06,81.89,83.18,1136.5,6.009,0.1350,274.7,220.32,85.950,96.300,141.60,73.55,28.41


## 7. Enregistrement du DataFrame propre

In [44]:
# Enregistrement du DataFrame nettoyé
final_df.to_csv('clean_commodities.csv')
print("DataFrame propre enregistré dans 'clean_commodities.csv'")

DataFrame propre enregistré dans 'clean_commodities.csv'


## 8. Résumé de la démarche

1. Nous avons chargé le fichier de données contenant les colonnes de dates et de prix pour différentes matières premières
2. Nous avons analysé la structure des données pour déterminer si les dates étaient alignées
3. Nous avons restructuré les données pour avoir un DataFrame propre avec une seule colonne de dates
4. Nous avons traité les valeurs manquantes par interpolation linéaire
5. Nous avons visualisé l'évolution des prix de quelques matières premières
6. Nous avons enregistré le DataFrame propre dans un nouveau fichier CSV

Cette approche permet d'obtenir un DataFrame propre et cohérent, prêt pour des analyses ultérieures.